In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical


In [2]:
# Load CIFAR-10 dataset
(itrain, ltrain), (itest, ltest) = cifar10.load_data()

In [3]:
# Preprocess the data
itrain = itrain / 255.0
itest = itest / 255.0
ltrain = to_categorical(ltrain)
ltest = to_categorical(ltest)


In [4]:
# Load pre-trained VGG16 model (excluding the top fully-connected layers)
basem = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [5]:
# Freeze the pre-trained layers
for layer in basem.layers:
   layer.trainable = False

In [6]:
# Create a new model on top
semodel = Sequential()
semodel.add(basem)
semodel.add(Flatten())
semodel.add(Dense(256, activation='relu'))
semodel.add(Dense(10, activation='softmax'))  # CIFAR-10 has 10 classes

In [10]:
semodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14,848,586
Trainable params: 133,898
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
# Compile the model
semodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
semodel.fit(itrain, ltrain, epochs=3, batch_size=32, validation_data=(itest, ltest))


Epoch 1/3
1563/1563 [==============================] - 1057s 674ms/step - loss: 1.3420 - accuracy: 0.5307 - val_loss: 1.2323 - val_accuracy: 0.5679
Epoch 2/3
1563/1563 [==============================] - 1022s 654ms/step - loss: 1.1700 - accuracy: 0.5917 - val_loss: 1.1836 - val_accuracy: 0.5817
Epoch 3/3
1563/1563 [==============================] - 1321s 845ms/step - loss: 1.1059 - accuracy: 0.6134 - val_loss: 1.1432 - val_accuracy: 0.5977


In [9]:
# Evaluate the model on test data
ltest, atest = semodel.evaluate(itest, ltest)
print("Test accuracy:", atest)

313/313 [==============================] - 235s 749ms/step - loss: 1.1432 - accuracy: 0.5977
Test accuracy: 0.5976999998092651
